In [1]:
# load libraries
import xarray as xr
import numpy as np

from argopy import DataFetcher as ArgoDataFetcher

from datetime import datetime, timedelta
import pandas as pd

In [2]:
# User defined functions:
def get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in,time_f):
    
    """Function to get argo data for a given lat,lon box (using Argopy), 
       and return a 2D array collection of vertical profile for the given region
    
    Parameters
    ----------
    llon : int
        left longitude
    rlon : int
        right longtidue
    ulat : int
        upper latitude
    llat : int
        lower latitude
    time_in : str/datetime object
        the start time of desired range, formatted Y-m-d
    time_f : str/datetime object
        the end time of desired range, formatted Y-m-d
    
    
    Returns
    ---------
    xarray
        The result is a xarray of the vertical profile for the given range and region.
        
    
    """

    ds_points = ArgoDataFetcher(src='erddap').region([llon,rlon, llat,ulat, depthmin, depthmax,time_in,time_f]).to_xarray()
    ds_profiles = ds_points.argo.point2profile()
    return ds_profiles

def spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end):
    """Function that gets the argo data for given latitude and longitude bounding box
       (using Argopy), and given start and end time range to return a 2D array collection of vertical
       profile for the given region and time frame
    
    Parameters
    ----------
    llon : int
        left longitude
    rlon : int
        right longtidue
    ulat : int
        upper latitude
    llat : int
        lower latitude
    time_in : str/datetime object
        the start time of desired range, formatted Y-m-d
    time_f : str/datetime object
        the end time of desired range, formatted Y-m-d
    
    
    Returns
    ---------
    xarray
        The result is a xarray of the vertical profile for the given range and region.
        
    
    """

    
    #step
    max_dt = timedelta(days = 10)
    
    if isinstance(time_start, str):
        time_start = datetime.strptime(time_start,"%Y-%m-%d")
    if isinstance(time_end, str):
        time_end = datetime.strptime(time_end,"%Y-%m-%d")
        
    if time_end - time_start <= max_dt:
        ds = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end)
        return ds
    else:
        early_end = time_start+max_dt
        ds = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,early_end)
        print("Retrived data from " + str(time_start) + " to " + str(early_end) + ", retreived " + str(len(ds.N_PROF)) + " profiles")
        ds2 = spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax, early_end,time_end)
        return xr.concat([ds,ds2],dim='N_PROF')


In [4]:
# Loop over 2014-2019 to get all winter months (jan and feb), and 
years=pd.date_range('2015-01-01','2020-01-01', 
              freq='Y').strftime("%Y").tolist()

# winter (jan-feb)
time_in=pd.date_range('2015-01-01','2019-01-01', 
              freq='YS').strftime("%Y-%m-%d").tolist()
time_f=[#'2014-03-01',
 '2015-03-01',
 '2016-03-01',
 '2017-03-01',
 '2018-03-01',
 '2019-03-01']

llon=-90;rlon=0
ulat=70;llat=0 
depthmin=0;depthmax=1400

for i,year in enumerate(years):
    ds = spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in[i],time_f[i])
    ds.to_netcdf(str('/home/jovyan/ohw20-proj-pyxpcm/data/winter'+str(year)+'.nc'))

Retrived data from 2015-01-01 00:00:00 to 2015-01-11 00:00:00, retreived 569 profiles
Retrived data from 2015-01-11 00:00:00 to 2015-01-21 00:00:00, retreived 560 profiles
Retrived data from 2015-01-21 00:00:00 to 2015-01-31 00:00:00, retreived 578 profiles
Retrived data from 2015-01-31 00:00:00 to 2015-02-10 00:00:00, retreived 580 profiles
Retrived data from 2015-02-10 00:00:00 to 2015-02-20 00:00:00, retreived 584 profiles
Retrived data from 2016-01-01 00:00:00 to 2016-01-11 00:00:00, retreived 609 profiles
Retrived data from 2016-01-11 00:00:00 to 2016-01-21 00:00:00, retreived 622 profiles
Retrived data from 2016-01-21 00:00:00 to 2016-01-31 00:00:00, retreived 611 profiles
Retrived data from 2016-01-31 00:00:00 to 2016-02-10 00:00:00, retreived 608 profiles
Retrived data from 2016-02-10 00:00:00 to 2016-02-20 00:00:00, retreived 612 profiles
Retrived data from 2017-01-01 00:00:00 to 2017-01-11 00:00:00, retreived 598 profiles
Retrived data from 2017-01-11 00:00:00 to 2017-01-21 0

In [5]:
# Loop over 2014-2019 to get all summer months (jul and aug), and 
years=pd.date_range('2015-01-01','2020-01-01', 
              freq='Y').strftime("%Y").tolist()

# summer (jul-aug)
time_in=[#'2014-07-01',
 '2015-07-01',
 '2016-07-01',
 '2017-07-01',
 '2018-07-01',
 '2019-07-01']

time_f=[#'2014-09-01',
 '2015-09-01',
 '2016-09-01',
 '2017-09-01',
 '2018-09-01',
 '2019-09-01']

llon=-90;rlon=0
ulat=70;llat=0 
depthmin=0;depthmax=1400

for i,year in enumerate(years):
    ds = spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in[i],time_f[i])
    ds.to_netcdf(str('/home/jovyan/ohw20-proj-pyxpcm/data/summer'+str(year)+'.nc'))

Retrived data from 2015-07-01 00:00:00 to 2015-07-11 00:00:00, retreived 633 profiles
Retrived data from 2015-07-11 00:00:00 to 2015-07-21 00:00:00, retreived 645 profiles
Retrived data from 2015-07-21 00:00:00 to 2015-07-31 00:00:00, retreived 623 profiles
Retrived data from 2015-07-31 00:00:00 to 2015-08-10 00:00:00, retreived 619 profiles
Retrived data from 2015-08-10 00:00:00 to 2015-08-20 00:00:00, retreived 616 profiles
Retrived data from 2015-08-20 00:00:00 to 2015-08-30 00:00:00, retreived 612 profiles
Retrived data from 2016-07-01 00:00:00 to 2016-07-11 00:00:00, retreived 746 profiles
Retrived data from 2016-07-11 00:00:00 to 2016-07-21 00:00:00, retreived 779 profiles
Retrived data from 2016-07-21 00:00:00 to 2016-07-31 00:00:00, retreived 746 profiles
Retrived data from 2016-07-31 00:00:00 to 2016-08-10 00:00:00, retreived 723 profiles
Retrived data from 2016-08-10 00:00:00 to 2016-08-20 00:00:00, retreived 713 profiles
Retrived data from 2016-08-20 00:00:00 to 2016-08-30 0